In [ ]:
import os, sys
import pandas as pd
import numpy as np
import datetime as dt
from pprint import pprint
import csv
import itertools as it
from prettytable import PrettyTable
# from nanrms import *
# from handover import *
# from mask import *
import matplotlib.pylab as plt
import seaborn as sns

pdir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # for jupyter-notebook
sys.path.insert(1, pdir)
from myutils import *

%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度
# plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

def set_data(df, mode='pcap', tz=0):
    if mode == 'pcap':
        df['seq'] = df['seq'].astype('Int32')
        # df['rpkg'] = df['rpkg'].astype('Int8')
        df['frame_id'] = df['frame_id'].astype('Int32')
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['xmit_time'] = pd.to_datetime(df['xmit_time'])
        df['arr_time'] = pd.to_datetime(df['arr_time'])
        df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
        df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
        df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
        df['lost'] = df['lost'].astype('boolean')
        df['excl'] = df['excl'].astype('boolean')
        df['latency'] = df['latency'].astype('float32')
    # E-UTRA & NR
    def nr_serv_cel(row):
        pos = row.serv_cel_pos
        if pos == 255:
            return 65535, -160, -50
        else:
            return row[f'PCI{pos}'], row[f'RSRP{pos}'], row[f'RSRQ{pos}']
    if mode == 'eutra':
        df = df.rename(columns={
            'RSRP(dBm)': 'RSRP',
            'RSRQ(dB)': 'RSRQ',
            'Serving Cell Index': 'serv_cel_index',
            'Number of Neighbor Cells': 'num_neigh_cels',
            'Number of Detected Cells': 'num_cels',
            }).reindex(['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                        'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0',
                        *df.columns.to_list()[df.columns.get_loc('PCI1'):]], axis=1)
        df.loc[df['serv_cel_index'] == '(MI)Unknown', 'serv_cel_index'] = '3_SCell'
        df['num_cels'] = df['num_neigh_cels'] + 1
        ## set dtypes
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df['type_id'] = df['type_id'].astype('category')
        df['serv_cel_index'] = df['serv_cel_index'].astype('category')
        df['EARFCN'] = df['EARFCN'].astype('Int32')
        df['NR_ARFCN'] = df['NR_ARFCN'].astype('Int32')
        df['num_cels'] = df['num_cels'].astype('UInt8')
        df['num_neigh_cels'] = df['num_neigh_cels'].astype('UInt8')
        df['serv_cel_pos'] = df['serv_cel_pos'].astype('UInt8')
        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP','RSRQ')):
                df[tag] = df[tag].astype('float32')
    if mode == 'nr':
        if df.empty:
            df = pd.DataFrame(columns=['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                                       'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0'])
        else:
            df = df.rename(columns={
                'Raster ARFCN': 'NR_ARFCN',
                'Serving Cell Index': 'serv_cel_pos',
                'Num Cells': 'num_cels',
                }).reindex(['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                            'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0',
                            *df.columns.to_list()[df.columns.get_loc('PCI1'):]], axis=1)
            df.loc[df['serv_cel_pos'] != 255, 'serv_cel_index'] = 'PSCell'
            df[['PCI','RSRP','RSRQ']] = df.apply(nr_serv_cel, axis=1, result_type='expand')
            df.loc[df['serv_cel_pos'] == 255, 'num_neigh_cels'] = df['num_cels']
            df.loc[df['serv_cel_pos'] != 255, 'num_neigh_cels'] = df['num_cels'] - 1
        ## set dtypes
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df['type_id'] = df['type_id'].astype('category')
        df['serv_cel_index'] = df['serv_cel_index'].astype('category')
        df['EARFCN'] = df['EARFCN'].astype('Int32')
        df['NR_ARFCN'] = df['NR_ARFCN'].astype('Int32')
        df['num_cels'] = df['num_cels'].astype('UInt8')
        df['num_neigh_cels'] = df['num_neigh_cels'].astype('UInt8')
        df['serv_cel_pos'] = df['serv_cel_pos'].astype('UInt8')
        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP','RSRQ')):
                df[tag] = df[tag].astype('float32')
    return df

In [ ]:
title = '8_Schemes'
datadir = "/Users/jackbedford/Desktop/MOXA/Code/data"
# datadir = "/home/wmnlab/D/database"
path = "./temp"
exps = {
    "2023-05-26/Bandlock_8_Schemes_Phone": ["#{:02d}".format(i+1) for i in range(6)],
}
_devices = [
    ["sm00", "sm01", "sm02", "sm03", "sm04", "sm05", "sm06", "sm07",],
]
_schemes = [
    ["All", "B3", "B7", "B8", "B3B7", "B3B8", "B7B8", "LTE",],
]

In [ ]:
# i = -1
# dfs_ul = []
# for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# # for date, traces in dates.items():
#     for trace in traces:
#         i += 1
#         dfs_ul.append([])
#         st, et = [], []
#         for j, (dev, schm) in enumerate(zip(devices, schemes)):
#             # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
#             data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
#             print(data, os.path.isfile(data))
#             df = pd.read_csv(data)
#             df = set_data(df)
#             dfs_ul[i].append(df.copy())
#             st.append(df['seq'].array[0])
#             et.append(df['seq'].array[-1])
#         st, et = max(st), min(et)
#         for j, (dev, schm) in enumerate(zip(devices, schemes)):
#             dfs_ul[i][j] = dfs_ul[i][j][(dfs_ul[i][j]['seq'] >= st) & (dfs_ul[i][j]['seq'] <= et)].reset_index(drop=True)
# print(len(dfs_ul))

# i = -1
# dfs_dl = []
# for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# # for date, traces in dates.items():
#     for trace in traces:
#         i += 1
#         dfs_dl.append([])
#         st, et = [], []
#         for j, (dev, schm) in enumerate(zip(devices, schemes)):
#             # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
#             data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
#             print(data, os.path.isfile(data))
#             df = pd.read_csv(data)
#             df = set_data(df)
#             dfs_dl[i].append(df.copy())
#             st.append(df['seq'].array[0])
#             et.append(df['seq'].array[-1])
#         st, et = max(st), min(et)
#         for j, (dev, schm) in enumerate(zip(devices, schemes)):
#             dfs_dl[i][j] = dfs_dl[i][j][(dfs_dl[i][j]['seq'] >= st) & (dfs_dl[i][j]['seq'] <= et)].reset_index(drop=True)
# print(len(dfs_dl))

# i = -1
# dfs_eutra = []
# for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# # for date, traces in dates.items():
#     for trace in traces:
#         i += 1
#         dfs_eutra.append([])
#         st, et = [], []
#         for j, (dev, schm) in enumerate(zip(devices, schemes)):
#             # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
#             dbpath = os.path.join(datadir, exp, dev, trace, 'data')
#             data = os.path.join(dbpath, [s for s in os.listdir(dbpath) if s.startswith('diag_log') and s.endswith('_ml1.csv') and not s.endswith('_nr_ml1.csv')][0])
#             print(data, os.path.isfile(data))
#             df = pd.read_csv(data)
#             df = set_data(df, mode='eutra', tz=8)
#             dfs_eutra[i].append(df.copy())
# print(len(dfs_eutra))

i = -1
dfs_nr = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_nr.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            dbpath = os.path.join(datadir, exp, dev, trace, 'data')
            data = os.path.join(dbpath, [s for s in os.listdir(dbpath) if s.startswith('diag_log') and s.endswith('_nr_ml1.csv')][0])
            print(data, os.path.isfile(data))
            df = pd.read_csv(data)
            df = set_data(df, mode='nr', tz=8)
            dfs_nr[i].append(df.copy())
print(len(dfs_nr))

i = -1
dfs_ho = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ho.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            dbpath = os.path.join(datadir, exp, dev, trace, 'data')
            data = os.path.join(dbpath, [s for s in os.listdir(dbpath) if s.startswith('diag_log') and s.endswith('_rrc.csv')][0])
            print(data, os.path.isfile(data))
            df = pd.read_csv(data)
            df, _, _ = mi_parse_ho(df, tz=8)
            df['ho_type0'] = df['ho_type'].astype('string')
            df.loc[df['cause'].notna(), 'ho_type0'] = df['ho_type'].astype('string') + '_' + df['cause'].astype('string')
            df['ho_type0'] = df['ho_type0'].astype('category')
            dfs_ho[i].append(df.copy())
print(len(dfs_ho))

In [ ]:
N = len(dfs_nr)
devices = _devices[0]

table = pd.DataFrame(columns=devices)
for i in range(N):
    check_list = ['O']*8
    for j, dev in enumerate(devices):
        df = dfs_ho[i][j].copy()
        if sum(df['cause'] == 'rlc-MaxNumRetx (2)') > 0:
            check_list[j] = '∆'
        df = dfs_nr[i][j].copy()
        if df.empty:
            check_list[j] = 'X'
    table.loc[len(table)] = check_list

display(table)
print('O: RF Module is always on.')
print('∆: RF Module was turned off midway. "SCG Failure: rlc-MaxNumRetx (2)" occurs!')
print('X: RF Module has never been turned on.')